# DataFrame LookUp Test

Go through the process of checking query speeds on a dataframe in which each row contains the entire article or in which each row is split by sentence. There are certainly quicker ways of doing some of the things in here. Notice for example that the dataframe is loaaded into memory and it is also copied in a couple of places (so that all of them are held in memory). For very large frames this should be batched.

## 1. Get some random data from a source .txt files of BBC articles

In [1]:
# get some mickey mouse data
import subprocess
dwnld = subprocess.run('wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip', shell=True, capture_output=True)
unzip = subprocess.run('unzip bbc-fulltext.zip', shell=True, capture_output=True)


## 2. Load this into a dataframe

In [5]:
# add it to my dataframe
import pandas as pd
df = pd.DataFrame( columns=['Filename','RawText','NormText'] )

# words I will "remove" when normalising text
normalised_removals = ['the','and','of','if','then','when','etc']

## function to load a file and append it to the frame
def load_file(df, fname):
    
    raw_string = ''
    norm_string = ''
    with open(fname) as infile:
        # note the following does need to read the entire file into memory first
        for line in infile: 
            if line in ['','\n','\r\n']:
                continue
            else:
                line = line.replace('\n','. ')
                raw_string += line
                for remove in normalised_removals:
                    line = line.replace(' '+remove+' ',' ')
                norm_string += line
                
    df = df.append( { 'Filename': fname,
                      'RawText': raw_string, 
                      'NormText': norm_string
                    }, ignore_index=True )
    return df

## function which takes the raw datafrarme and return a copy with a 
## sub-index split by sentence
def split_at_sentence(df):
    sub_df = df.copy()
    sub_df['RawText'] = sub_df['RawText'].str.split('.')
    sub_df['NormText'] = sub_df['NormText'].str.split('.')
    sub_df['SentenceID'] = sub_df['RawText'].apply(len).apply(range).apply(list)
    sub_df = sub_df.explode(['SentenceID','RawText','NormText'])
    sub_df['FileID'] = list(sub_df.index.values)
    sub_df = sub_df.set_index(['FileID','SentenceID'])
    return sub_df

## loop files and append them to the df
import glob
for file in glob.glob('bbc/business/*.txt'):
    df = load_file(df, file)

## split the df into sentences
sent_df = split_at_sentence(df)


In [6]:
## now check look up speeds

import time

# function to time a process
def howlong( func, *args ):
    start = time.time()
    ret = func(*args)
    end = time.time()
    took = end -  start
    return took, ret

# function to perform the search of a word in "NormText"
# and return the resulting frame of matching results
def search(df, word):
    res = df['NormText'].str.contains(search_word, na=True)
    return df[res]

# function to process the return (i.e. return the "RawText" sentence
# which is relevant)
def process_return(df, word, is_split=True):
    # we want to return the relevent sentences
    # if the dataframe is already split by sentence this 
    # is trivial
    if is_split:
        return df['RawText']
    # otherwise we now have to find the sentence it appeared in
    # the quickest way I can think to do this is do what we have 
    # already done
    else:
        split_df = df.copy()
        split_df['RawText'] = split_df['RawText'].str.split('.')
        split_df['NormText'] = split_df['NormText'].str.split('.')
        split_df = split_df.explode(['RawText','NormText'])
        split_df = search(split_df, word)
        return split_df['RawText']

# function to amalgamate the search and process functions into one
# for timing audit
def search_and_process(df, word, is_split):
    resdf = search(df, word)
    return process_return(resdf, word, is_split)




In [7]:
## Now actually perform the search
search_word = 'market'

t1, _ = howlong(search, df, search_word)
t2, _ = howlong(search, sent_df, search_word)
t3, resdf1 = howlong(search_and_process, df, search_word, False)
t4, resdf2 = howlong(search_and_process, sent_df, search_word, True)

print(f'Original df had {len(df)} entries')
print(f'Curated sent_df has {len(sent_df)} entries')
print('')
print(f'Raw search time in original df = {t1*1000:4.1f}ms')
print(f'Raw search time in curated  df = {t2*1000:4.1f}ms')
print(f'Search and process time in original df = {t3*1000:4.1f}ms')
print(f'Search and process time in curated  df = {t4*1000:4.1f}ms')



Original df had 510 entries
Curated sent_df has 12725 entries

Raw search time in original df =  2.1ms
Raw search time in curated  df = 10.6ms
Search and process time in original df = 14.5ms
Search and process time in curated  df =  6.7ms
